In [25]:
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\original_data\220915_mtstaygold_cos7_ZEISS_fl\labels3f.csv'
imagefile= r'C:\Users\roumba\Documents\Software\deep-events\original_data\220915_mtstaygold_cos7_ZEISS_fl\220915_cos7_mitostaygold_GFP_3.ome.tif'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from scipy import signal
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
import myfunctions
from myfunctions import event_separation , image_crop_save_gauss , get_gaussian , image_crop_save , image_crop_negative

img = Image.open(imagefile)
datacsv = pd.read_csv(csvfile)
sigma_trial=(8,8)
size_trial=(2048,2048)


In [26]:
from pathlib import Path
path = Path(imagefile)
file_details = path.stem
date, cell_type, dye, bf_fl, index  = file_details.split('_')
input_name= f'points_{date}_{cell_type}_{bf_fl}_{index}.tiff'
output_name= f'image_{date}_{cell_type}_{bf_fl}_{index}'

In [27]:
points_of_interest= np.zeros((100, 2048, 2048))

for row_number in tqdm(range(0, len(datacsv))):
    framenumber_in_row= int(datacsv.loc[row_number, 'axis-0'])
    fission_ycoord = int(datacsv.loc[row_number, 'axis-1'])
    fission_xcoord = int(datacsv.loc[row_number, 'axis-2'])
    fission_coords=(fission_ycoord,fission_xcoord)
    gaussian_points=get_gaussian(fission_coords,sigma_trial,size_trial)                                     #gets gaussian points at a single frame
    gaussian_points = gaussian_points.numpy()                                                               #convers tensor into numpy array
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by the max 
    gaussian_points[gaussian_points < 0.1] = 0                                                              #sets background to zero
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by max again
    points_of_interest[framenumber_in_row] = points_of_interest[framenumber_in_row] + gaussian_points       #adds the gaussian intensity in the empty file 


   
imageio.mimwrite(input_name, (points_of_interest*254).astype(np.uint8))

100%|██████████| 32/32 [00:07<00:00,  4.53it/s]


In [28]:
## CROPS IMAGES ##
list1=event_separation(datacsv)
l=len(list1)
image_crop_save(l,list1, datacsv, img, output_name)

In [29]:
## CROPS GAUSS IMAGES ##
gauss=r'C:\Users\roumba\Documents\Software\deep-events\points_220915_cos7_GFP_2.ome.tiff'
gauss_image=Image.open(gauss)
image_crop_save_gauss(l,list1, datacsv, gauss_image, output_name)

In [30]:
## PRODUCES NEGATIVE IMAGE CROPS ##
image_crop_negative(l,list1, datacsv, img, output_name)